Урок 3. Инструменты работы и визуализации ч.2

Условие: есть набор данных о продажах продуктов с информацией о дате продаж, категории продукта, количестве и выручке от продаж.

Используя Apache Spark, загрузите предоставленный набор данных в DataFrame (пример данных ниже).

С использованием оконных функций, рассчитайте среднее выручки от продаж для каждой категории продукта.

Примените операцию pivot для того, чтобы преобразовать полученные данные таким образом, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни, а значениями были средние значения выручки от продаж за соответствующий день

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import col, avg

In [29]:
spark = SparkSession.builder.appName("SalesInfo").getOrCreate()


In [30]:
# Создаем Spark DataFrame на основе данных задания
data = [
   ("2023-11-20", "Electronics", 100, 12000),
   ("2023-11-21", "Electronics", 110, 13000),
   ("2023-11-22", "Electronics", 105, 12500),
   ("2023-11-20", "Clothing", 300, 15000),
   ("2023-11-21", "Clothing", 280, 14000),
   ("2023-11-22", "Clothing", 320, 16000),
   ("2023-11-20", "Books", 150, 9000),
   ("2023-11-21", "Books", 200, 12000),
   ("2023-11-22", "Books", 180, 10000)
]
columns = ["date", "category", "quantity", "revenue"]
spark_df = spark.createDataFrame(data, columns)
spark_df.show()

+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+



In [34]:
# Определяем окно и рассчитываем среднее выручки от продаж за день для каждой категории продукта
window_cat_avg_sales = Window.partitionBy(["category", "date"])
spark_df = spark_df.withColumn("avg_revenue", avg("revenue").over(window_cat_avg_sales))
spark_df.select(["category", "date", "avg_revenue"]).distinct().show()

+-----------+----------+-----------+
|   category|      date|avg_revenue|
+-----------+----------+-----------+
|      Books|2023-11-20|     9000.0|
|      Books|2023-11-21|    12000.0|
|      Books|2023-11-22|    10000.0|
|   Clothing|2023-11-20|    15000.0|
|   Clothing|2023-11-21|    14000.0|
|   Clothing|2023-11-22|    16000.0|
|Electronics|2023-11-20|    12000.0|
|Electronics|2023-11-21|    13000.0|
|Electronics|2023-11-22|    12500.0|
+-----------+----------+-----------+



In [32]:
# Применяем операцию pivot, чтобы в качестве строк были категории продуктов, в качестве столбцов были дни,
# а значениями были средние значения выручки от продаж за соответствующий день
pivot_df = spark_df.groupBy("category").pivot("date").agg(avg("revenue")).orderBy("category")
pivot_df.show()

+-----------+----------+----------+----------+
|   category|2023-11-20|2023-11-21|2023-11-22|
+-----------+----------+----------+----------+
|      Books|    9000.0|   12000.0|   10000.0|
|   Clothing|   15000.0|   14000.0|   16000.0|
|Electronics|   12000.0|   13000.0|   12500.0|
+-----------+----------+----------+----------+



In [ ]:
spark.stop()